### Импорт библиотек для работы с бд и дф

In [68]:
import psycopg2
import pandas as pd
import sqlalchemy

### Кредиты для коннекта к бд


In [69]:
conn = psycopg2.connect(host="127.0.0.1", port="5433", database="online_store", 
                            user="admin", password="root")

### Создание таблицы и наполнение ее данными

In [70]:
cur = conn.cursor()
cur.execute('''
            CREATE TABLE statistic(
            player_name VARCHAR(100) NOT NULL,
            player_id INT NOT NULL,
            year_game SMALLINT NOT NULL CHECK (year_game > 0),
            points DECIMAL(12,2) CHECK (points >= 0),
            PRIMARY KEY (player_name,year_game)
            );
            ''')

cur.execute('''
           INSERT INTO
            statistic(player_name, player_id, year_game, points)
            VALUES
            ('Mike',1,2018,18),
            ('Jack',2,2018,14),
            ('Jackie',3,2018,30),
            ('Jet',4,2018,30),
            ('Luke',1,2019,16),
            ('Mike',2,2019,14),
            ('Jack',3,2019,15),
            ('Jackie',4,2019,28),
            ('Jet',5,2019,25),
            ('Luke',1,2020,19),
            ('Mike',2,2020,17),
            ('Jack',3,2020,18),
            ('Jackie',4,2020,29),
            ('Jet',5,2020,27);
            ''')

cur.execute('''
           SELECT 
           player_name, player_id, year_game, points
           from statistic
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['player_name', 'player_id', 'year_game', 'points'])

,player_name,player_id,year_game,points
0,Mike,1,2018,18.00
1,Jack,2,2018,14.00
2,Jackie,3,2018,30.00
3,Jet,4,2018,30.00
4,Luke,1,2019,16.00
5,Mike,2,2019,14.00
6,Jack,3,2019,15.00
7,Jackie,4,2019,28.00
8,Jet,5,2019,25.00
9,Luke,1,2020,19.00


### запрос суммы очков с группировкой и сортировкой по годам

In [71]:
cur.execute('''
           SELECT 
           year_game,
           sum(points) as sum_points
           from statistic
           group by year_game
           order by year_game
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['year_game', 'sum_points'])

,year_game,sum_points
0,2018,92.00
1,2019,98.00
2,2020,110.00


### CTE Common Table Expressions 
### Используя функцию LAG вывести кол-во очков по всем игрокам за текущий код и за предыдущий

In [72]:
cur.execute('''
           with cte_sum as (
               SELECT 
               year_game,
               sum(points) as sum_points
               from statistic
               group by year_game
               order by year_game
           )
           select 
               year_game, 
               sum_points as cur_y,
               LAG(sum_points) over (order by year_game) as lst_y
           from cte_sum
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['year_game', 'cur_y','lst_y'])

,year_game,cur_y,lst_y
0,2018,92.00,None
1,2019,98.00,92.00
2,2020,110.00,98.00


In [73]:
conn.rollback()
conn.close()